In [17]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.stats import pearsonr

from modules.data_fetcher import download_historical_data
from modules.indicators import compute_lagging_regression

In [2]:
symbol= 'BTC-USDT'

df_BTC = download_historical_data(symbol,'1hour').iloc[-3000:]
df_BTC["Return"] = df_BTC.Close.pct_change()
df_BTC.dropna(inplace=True)
print(df_BTC.shape)
df_BTC.head()

(2999, 8)


,Timestamp,Open,Close,High,Low,Amount,Volume,Return
Date,,,,,,,,
2022-10-25 18:00:00,1.666714e+09,19748.5,19812.1,19842.6,19714.5,1138.714141,2.252093e+07,0.003231
2022-10-25 19:00:00,1.666717e+09,19812.0,20117.5,20194.0,19810.3,2755.522173,5.516368e+07,0.015415
2022-10-25 20:00:00,1.666721e+09,20117.5,20272.9,20409.8,20072.9,2243.849539,4.545751e+07,0.007725
2022-10-25 21:00:00,1.666724e+09,20270.5,20274.1,20414.0,20202.0,1308.519245,2.656544e+07,0.000059
2022-10-25 22:00:00,1.666728e+09,20272.1,20186.6,20296.1,20010.3,1745.241177,3.509599e+07,-0.004316


# Predict crypto from BTC

In [26]:
symbol_to_predict= 'AVAX-USDT'

df_to_predict= download_historical_data(symbol_to_predict,'1hour')#.iloc[-3000:]
df_to_predict["Return"] = df_to_predict.Close.pct_change()
df_to_predict.dropna(inplace=True)
print(df_to_predict.shape)
df_to_predict.head()

(17381, 8)


,Timestamp,Open,Close,High,Low,Amount,Volume,Return
Date,,,,,,,,
2021-03-05 12:00:00,1.614942e+09,24.382,24.927,24.927,24.382,644.195305,15884.961739,0.022059
2021-03-05 13:00:00,1.614946e+09,24.938,24.734,25.465,24.500,1803.342134,44441.071379,-0.007743
2021-03-05 14:00:00,1.614949e+09,25.000,25.445,25.445,24.892,285.892382,7190.862738,0.028746
2021-03-05 15:00:00,1.614953e+09,25.271,25.011,26.000,24.967,501.201474,12800.578898,-0.017056
2021-03-05 16:00:00,1.614956e+09,25.185,24.506,25.275,24.506,375.273473,9331.933770,-0.020191


In [27]:
merged = pd.merge(df_BTC, df_to_predict, left_index=True,right_index=True,suffixes=('',f'_{symbol_to_predict}'))[['Close',f'Close_{symbol_to_predict}']]

merged.head()

,Close,Close_AVAX-USDT
Date,,
2022-10-25 18:00:00,19812.1,16.778
2022-10-25 19:00:00,20117.5,16.954
2022-10-25 20:00:00,20272.9,16.856
2022-10-25 21:00:00,20274.1,16.818
2022-10-25 22:00:00,20186.6,16.535


In [28]:
SHIFT = 20
fig = make_subplots(
    rows=1, cols=1
)

fig.add_trace(
    go.Scatter(
        name="Not shifted",
        mode='markers',
        x=merged["Close"],
        y=merged[f'Close_{symbol_to_predict}'],
        opacity=0.5,
        marker=dict(
            color='Red',
            size=7,
        ),
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        name="Shifted",
        mode='markers',
        x=merged["Close"].iloc[:-SHIFT],
        y=merged[f'Close_{symbol_to_predict}'].iloc[SHIFT:],
        opacity=0.5,
        marker=dict(
            color='blue',
            size=7,
        ),
    ),
    row=1,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="Comparing relation between pair to pair data and shifted data",
)



In [29]:
print(pearsonr(merged["Close"].iloc[:-SHIFT],merged[f'Close_{symbol_to_predict}'].iloc[SHIFT:]))

PearsonRResult(statistic=0.9402776092485293, pvalue=0.0)
